<a href="https://colab.research.google.com/github/OsvaldoUfla/Atividade-com-GPU-12-08-24/blob/main/Atividade_openacc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Atividade programação GPU com OpenACC
Osvaldo Rorigues de Faria Junior - 201911203

# Soma de vetores
Onde ocorre paralelismo:   
Dentro da região #pragma acc data, a função sumArraysOnHost é chamada duas vezes, uma vez para somar h_A com h_B, e outra para somar d_A com d_B. Ambas as operações são realizadas em paralelo, como definido na função sumArraysOnHost.

In [91]:
%%writefile soma_vetor.c
#include <stdlib.h>
#include <time.h>
#include <stdio.h>
#include <openacc.h>

void sumArraysOnHost(float *A, float *B, float *C, const int N)
{
    #pragma acc parallel loop
    for (int idx = 0; idx < N; idx++)
    {
        C[idx] = A[idx] + B[idx];
    }
}

void initialData(float *ip, int size)
{
    // gerar semente diferente para número aleatório
    time_t t;
    srand((unsigned) time(&t));

    for (int i = 0; i < size; i++)
    {
        ip[i] = (float)(rand() & 0xFF) / 10.0f;
    }

    return;
}

void initialDatasequencial(float *ip,float *ip2, int size)
{
    for (int i = 0; i < size; i++)
    {
        ip[i] = i;
        ip2[i] = size-i;
    }

    return;
}

int main(int argc, char **argv)
{
    int nElem = 1024;
    size_t nBytes = nElem * sizeof(float);

    float *h_A, *h_B, *h_C;
    float *d_A, *d_B, *d_C;
    h_A = (float *)malloc(nBytes);
    h_B = (float *)malloc(nBytes);
    h_C = (float *)malloc(nBytes);
    d_A = (float *)malloc(nBytes);
    d_B = (float *)malloc(nBytes);
    d_C = (float *)malloc(nBytes);


    initialData(h_A, nElem);
    initialData(h_B, nElem);
    initialDatasequencial(d_A,d_B, nElem);

    // Copiar dados para o dispositivo
    #pragma acc data copyin(h_A[0:nElem], h_B[0:nElem]) copyout(h_C[0:nElem])
    {
        sumArraysOnHost(h_A, h_B, h_C, nElem);
        sumArraysOnHost(d_A, d_B, d_C, nElem);
    }

    // Verificar os resultados
    for (int i = 0; i < 10; i++)
    {
        printf("C[%d] = %f\n", i, h_C[i]);
    }

    // Verificar se resultados estão corretos
    for (int i = 0; i < 10; i++)
    {
        printf("C[%d] = %f\n", i, d_C[i]);
    }

    free(h_A);
    free(h_B);
    free(h_C);
    free(d_A);
    free(d_B);
    free(d_C);

    return(0);
}


Overwriting soma_vetor.c


In [92]:
!gcc -fopenacc soma_vetor.c -o soma_vetor

In [93]:
!./soma_vetor

C[0] = 15.600000
C[1] = 12.200000
C[2] = 28.400000
C[3] = 17.000000
C[4] = 1.800000
C[5] = 14.400000
C[6] = 25.600000
C[7] = 16.799999
C[8] = 14.400000
C[9] = 34.799999
C[0] = 1024.000000
C[1] = 1024.000000
C[2] = 1024.000000
C[3] = 1024.000000
C[4] = 1024.000000
C[5] = 1024.000000
C[6] = 1024.000000
C[7] = 1024.000000
C[8] = 1024.000000
C[9] = 1024.000000


# Soma de matrizes
Onde ocorre paralelismo:   
O paralelismo no código ocorre dentro da função matAdd, onde a soma das matrizes é distribuída entre múltiplos threads utilizando a diretiva #pragma acc parallel loop. Isso permite que cada elemento das matrizes seja somado em paralelo, aproveitando a capacidade de múltiplos núcleos de CPU ou de uma GPU para acelerar a operação.

In [94]:
%%writefile soma_matriz.c
#include <stdio.h>
#include <stdlib.h>
#include <openacc.h>

void matAdd(int width, int height, const float* A, const float* B, float* C)
{
    #pragma acc parallel loop collapse(2) copyin(A[0:width*height], B[0:width*height]) copyout(C[0:width*height])
    for (int i = 0; i < height; i++)
    {
        for (int j = 0; j < width; j++)
        {
            int index = i * width + j;
            C[index] = A[index] + B[index];
        }
    }
}

int main()
{
    int width = 1000;
    int height = 100;

    int numElements = width * height;

    float* A = (float*)calloc(numElements, sizeof(float));
    float* B = (float*)calloc(numElements, sizeof(float));
    float* C = (float*)calloc(numElements, sizeof(float));

    if (A == NULL || B == NULL || C == NULL)
    {
        fprintf(stderr, "Memory allocation failed\n");
        return 1;
    }

    srand(1214134);
    for (int i = 0; i < numElements; i++)
    {
        A[i] = (float)rand() / (float)(RAND_MAX + 1.0);
        B[i] = (float)rand() / (float)(RAND_MAX + 1.0);
    }

    matAdd(width, height, A, B, C);

    for (int i = 0; i < (height < 5 ? height : 5); i++)
    {
        for (int j = 0; j < (width < 5 ? width : 5); j++)
        {
            int index = i * width + j;
            printf("%3.2f + %3.2f = %3.2f;\t", A[index], B[index], C[index]);
        }
        printf("...\n");
    }
    printf("...\n");

    free(A);
    free(B);
    free(C);

    return 0;
}


Overwriting soma_matriz.c


In [95]:
!gcc -fopenacc soma_matriz.c -o soma_matriz

In [96]:
!./soma_matriz

0.47 + 0.76 = 1.23;	0.11 + 0.65 = 0.77;	0.13 + 0.70 = 0.83;	0.22 + 0.13 = 0.35;	0.96 + 0.03 = 0.99;	...
0.24 + 0.24 = 0.48;	0.59 + 0.11 = 0.70;	0.55 + 0.88 = 1.43;	0.43 + 0.05 = 0.48;	0.30 + 0.57 = 0.87;	...
0.21 + 0.77 = 0.98;	0.21 + 0.86 = 1.07;	0.92 + 0.10 = 1.02;	0.64 + 0.03 = 0.67;	0.45 + 0.01 = 0.46;	...
0.16 + 0.23 = 0.39;	0.73 + 0.08 = 0.81;	0.19 + 0.11 = 0.30;	0.65 + 0.41 = 1.06;	0.54 + 0.41 = 0.95;	...
0.43 + 0.11 = 0.54;	0.66 + 0.23 = 0.89;	0.64 + 0.05 = 0.68;	0.51 + 0.51 = 1.02;	0.31 + 0.05 = 0.36;	...
...


# Crivo OpenACC   
Onde ocorre paralelismo:    
O paralelismo ocorre nos dois laços principais da função crivo_array:    
    1 - Inicialização do vetor a: Cada índice do vetor é atribuído simultaneamente em diferentes threads.    
    2 - Marcação dos múltiplos de k: A marcação dos múltiplos de um número k é feita em paralelo, permitindo que várias partes do vetor sejam processadas ao mesmo tempo.     
Essa abordagem paralela pode levar a uma grande melhoria no desempenho em sistemas com múltiplos núcleos ou GPUs, especialmente quando N é grande, pois a execução paralela reduz o tempo necessário para percorrer e processar o vetor a.

In [97]:
%%writefile crivo.c
#include <openacc.h>
#include <stdio.h>
#include <stdlib.h>

void crivo_array(int *a, int N) {

    int k = 2;

    #pragma acc parallel loop
    for (int idx = 0; idx < N; idx++) {
        a[idx] = idx;
    }

    while (k * 2 <= N) {
        #pragma acc parallel loop
        for (int idx = 0; idx < N; idx++) {
            if (((a[idx] % k) == 0) && (a[idx] > 0) && (a[idx] != k)) {
                a[idx] = -1; // marco o elemento na lista
            }
        }

        k++;
        while (a[k] < 0 && k < N) {
            k++;
        }
    }
}

int main(void) {

    int *a;
    const int N = 1000; // Numero de elementos da lista

    size_t size = N * sizeof(int); // determinando o tamanho do vetor de inteiros

    a = (int*)malloc(size); // alocando espaço na memoria da CPU

    // faz crivo
    #pragma acc data copy(a[0:N])
    {
        crivo_array(a, N);
    }
    // fim do crivo

    printf("São primos os números entre 2 e %d\n", N);
    for (int i = 2; i < N; i++) {
        if (a[i] > 0)
            printf("%d  --  ", a[i]); // imprimo somente os números não marcados (primos)
    }

    free(a); // Liberando memoria da CPU
    return 0;
}



Overwriting crivo.c


In [98]:
!gcc -fopenacc crivo.c -o crivo

In [99]:
!./crivo

São primos os números entre 2 e 1000
2  --  3  --  5  --  7  --  11  --  13  --  17  --  19  --  23  --  29  --  31  --  37  --  41  --  43  --  47  --  53  --  59  --  61  --  67  --  71  --  73  --  79  --  83  --  89  --  97  --  101  --  103  --  107  --  109  --  113  --  127  --  131  --  137  --  139  --  149  --  151  --  157  --  163  --  167  --  173  --  179  --  181  --  191  --  193  --  197  --  199  --  211  --  223  --  227  --  229  --  233  --  239  --  241  --  251  --  257  --  263  --  269  --  271  --  277  --  281  --  283  --  293  --  307  --  311  --  313  --  317  --  331  --  337  --  347  --  349  --  353  --  359  --  367  --  373  --  379  --  383  --  389  --  397  --  401  --  409  --  419  --  421  --  431  --  433  --  439  --  443  --  449  --  457  --  461  --  463  --  467  --  479  --  487  --  491  --  499  --  503  --  509  --  521  --  523  --  541  --  547  --  557  --  563  --  569  --  571  --  577  --  587  --  593  --  599  --  601  --  60